In [1]:
!pip install pandas

In [2]:
import pandas as pd
import sqlite3
from math import radians, cos, sin, asin, sqrt

In [4]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers.
    return int(c * r)

In [18]:
def apply_haversine(x, lat, lon):
    """Calculate distance based on 2 coordinates"""
    if x.Visited==False:
        distance = haversine(lat, lon, x.latitude, x.longitude)
    else:
        distance = None
    return distance

In [19]:
geocodes_df.loc[639, ['latitude', 'longitude']].values

array([51.26150131225586, 9.449399948120117], dtype=object)

In [20]:
lati, longi = 53.474370, 21.749222

In [40]:
beers_df = pd.read_csv('beers.csv', index_col=0)
breweries_df = pd.read_csv('breweries.csv', index_col=0)
geocodes_df = pd.read_csv('geocodes.csv', index_col=0)
geocodes_df['Visited']=False

start_lat = lati
start_lon = longi
lat = start_lat
lon = start_lon
haversine_df = pd.DataFrame()
travel_list = []
current_distance = 0
total_distance = 0
isComplete = True

while isComplete:
    haversine_df = geocodes_df.apply(apply_haversine, args=(lat, lon), axis=1)
    min_id = haversine_df.idxmin(skipna=True)
    lat, lon = geocodes_df.loc[min_id, ['latitude', 'longitude']]
    distance_to_start = haversine(start_lat, start_lon, lat, lon)
    current_distance+=haversine_df[min_id]
    total_distance = current_distance + distance_to_start
    if total_distance < 2000:
        travel_list.append(min_id)
        geocodes_df.loc[min_id, 'Visited']=True
    else:
        lat, lon = geocodes_df.loc[travel_list[-1], ['latitude', 'longitude']]
        distance_to_start = haversine(start_lat, start_lon, lat, lon)
        current_distance-=haversine_df[min_id]
        total_distance = current_distance + distance_to_start
        isComplete = False
        

print(travel_list, total_distance)

[287, 288, 936, 902, 210, 198, 220, 1054] 1995.0
